Teste Prático para Engenheiros de Dados - Itaú RTDA

In [119]:
# Importa libs necessárias

import pandas as pd
import boto3
import io

from pyspark.sql import SparkSession, functions as f
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType

from dotenv import load_dotenv
from os import getenv
load_dotenv()

import requests as req
import json

from datetime import datetime

#import pytest
import unittest


1. ETL e Manipulação de Dados

Utilize o arquivo sales_data.csv e:
Limpe os dados removendo linhas duplicadas e tratando valores ausentes.
Transforme o valor da venda de uma moeda fictícia para USD usando a taxa de conversão de 1 FICT = 0.75 USD.
Carregue os dados limpos e transformados em um banco de dados relacional.

In [120]:

# Implementação estrutural

# Passo 1: Extração e Limpeza dos Dados
# Carrega o arquivo CSV
df_sales_raw = pd.read_csv('input/sales_data.csv')

# Conta o número de linhas após as transformações
print("O dataframe original tem",len(df_sales_raw.index),"linhas.")

# Remove linhas duplicadas
df_sales_refined = df_sales_raw.drop_duplicates()

# Tratar valores ausentes (caso existam)
# Substitui valores ausentes por zero.
df_sales_refined = df_sales_refined.fillna(0)

# Passo 2: Transformação dos Dados
# Converter o valor da venda de moeda fictícia (FICT) para USD usando a taxa de conversão de 1 FICT = 0.75 USD
df_sales_refined['usd_sale_value'] = df_sales_refined['sale_value'] * 0.75

# Conta o número de linhas após as transformações
print("O novo dataframe tem",len(df_sales_refined.index),"linhas.")

#df_sales_refined.to_csv("output/struct_sales_data.csv", index=False)

json_sales_refined = df_sales_refined.to_json()

aws_access_key_id = getenv("AWS_ACCESS_KEY")
aws_secret_access_key = getenv("AWS_SECRET_KEY")
    
s3 = boto3.client(
    's3'
    , aws_access_key_id=aws_access_key_id
    , aws_secret_access_key=aws_secret_access_key
    )

bucket_name = 'bucket-teste-iam'
file_name = 'json_sales_refined.json'

s3.put_object(Bucket=bucket_name, Key=file_name, Body=json_sales_refined)

O dataframe original tem 40 linhas.
O novo dataframe tem 40 linhas.


{'ResponseMetadata': {'RequestId': 'SHWAAEC74RQ3689R',
  'HostId': 'sXp61PSBR8iu+gITUeFwVBLHC9sN/8qvBspE0lQKDGq+7e0QRwI92L6T0bD/wQjjul+pflY26Ww=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sXp61PSBR8iu+gITUeFwVBLHC9sN/8qvBspE0lQKDGq+7e0QRwI92L6T0bD/wQjjul+pflY26Ww=',
   'x-amz-request-id': 'SHWAAEC74RQ3689R',
   'date': 'Mon, 07 Aug 2023 16:18:26 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5fbc678193a6c41a0b5cf6dd103a82e1"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5fbc678193a6c41a0b5cf6dd103a82e1"',
 'ServerSideEncryption': 'AES256'}

### 2. Análise com Apache Spark(utilize PySpark ou Spark)

#### Descrição

Dado um conjunto fictício de logs `website_logs.csv`:

- Identifique as 10 páginas mais visitadas.
- Calcule a média de duração das sessões dos usuários.
- Determine quantos usuários retornam ao site mais de uma vez por semana.

In [121]:
# Cria sessão Spark

spark = (SparkSession.builder
                     .master("local[1]")
                     .getOrCreate()
         )

In [122]:
# Importa o Dataframe
schema = StructType(
    [
          StructField("user_id", StringType())
         ,StructField("page_url", StringType())
         ,StructField("session_duration", FloatType())
         ,StructField("date", DateType())
    ]
)

df_website_access = spark.read.csv("input/website_logs.csv", header=True, schema=schema)
df_website_access.show()
df_website_access.printSchema()

+-------+-----------------+----------------+----------+
|user_id|         page_url|session_duration|      date|
+-------+-----------------+----------------+----------+
|  10001|    homepage.html|            15.0|2023-07-25|
|  10002|product_page.html|           120.0|2023-07-25|
|  10003|    checkout.html|            45.0|2023-07-25|
|  10004|     contact.html|            20.0|2023-07-25|
|  10005|    homepage.html|            10.0|2023-07-25|
|  10006|product_page.html|            95.0|2023-07-25|
|  10007|        blog.html|           150.0|2023-07-26|
|  10008|    homepage.html|            25.0|2023-07-26|
|  10009|product_page.html|            85.0|2023-07-26|
|  10010|    checkout.html|            50.0|2023-07-26|
|  10011|         faq.html|            35.0|2023-07-27|
|  10012|    homepage.html|            12.0|2023-07-27|
|  10013|product_page.html|           110.0|2023-07-27|
|  10014|    checkout.html|            60.0|2023-07-27|
|  10015|        blog.html|           160.0|2023

In [123]:
# 2.1 - Identifique as 10 páginas mais visitadas.

df_website_access.groupby("page_url")  \
    .count()                 \
    .sort(f.col("count")
          .desc())              \
    .show(10)

+-----------------+-----+
|         page_url|count|
+-----------------+-----+
|    homepage.html|   51|
|product_page.html|   46|
|    checkout.html|   35|
|     contact.html|   19|
|        blog.html|   18|
|         faq.html|   14|
|    about_us.html|    1|
+-----------------+-----+



In [124]:
# 2.2 Calcule a média de duração das sessões dos usuários.

df_website_access.select(f.avg("session_duration").alias("Duração média das sessões")) \
    .show(10)

+-------------------------+
|Duração média das sessões|
+-------------------------+
|        60.84239130434783|
+-------------------------+



In [125]:
# 2.3 Determine quantos usuários retornam ao site mais de uma vez por semana.

# Cria uma coluna que identifica a semana e o ano
df_website_access = df_website_access.withColumn("year_week",f.concat_ws("-",f.year("date"),f.weekofyear("date")))

# Cria um Dataframe com os usuários que retornam com mais de 1 acesso semanal
df_weekly_website_access = df_website_access.groupBy("user_id","year_week").count().filter("count > 1")

df_returning_users = df_weekly_website_access.select("user_id").distinct().count()
output_message = f"{df_returning_users} usuários retornam ao site mais de uma vez por semana."
print(output_message)


21 usuários retornam ao site mais de uma vez por semana.


### 3. Desenho de Arquitetura

#### Descrição

Proponha uma arquitetura em AWS para coletar dados de diferentes fontes:

- Desenhe um sistema para coletar dados de uma API.
- Processe esses dados em tempo real.
- Armazene os dados para análise futura.

In [126]:
from diagrams import Cluster, Diagram
from diagrams.aws.compute import Lambda
from diagrams.aws.network import APIGateway
from diagrams.aws.analytics import KinesisDataStreams, Redshift
from diagrams.aws.storage import S3

with Diagram("Coleta e Processamento de Dados", show=False):
    with Cluster("Coleta de Dados"):
        api_gateway = APIGateway("API Gateway")
        data_source = api_gateway >> Lambda("Processador de Dados")
        
    with Cluster("Processamento em Tempo Real"):
        kinesis_stream = KinesisDataStreams("Kinesis Stream")
        data_processor = Lambda("Processamento\nem Tempo Real")
        data_source >> data_processor >> kinesis_stream

    with Cluster("Armazenamento e Análise"):
        s3_bucket = S3("Bucket S3")
        redshift = Redshift("Amazon Redshift")
        kinesis_stream >> s3_bucket
        s3_bucket >> redshift

### 4. Codificação

#### Descrição

Escreva um script Python para:

- Se conectar à [API de previsão do tempo OpenWeatherMap](https://openweathermap.org/api).
- Coletar dados dessa API para uma cidade de sua escolha.
- Armazenar os dados coletados em um banco de dados relacional ou NoSQL(de sua escolha).

In [127]:
API_KEY = getenv("OPEN_WHEATER_API_KEY")
cidade = "miami"
link = f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={API_KEY}&lang=pt_br"

with req.get(link) as r:
    if r.status_code != 200:
        print("Erro ao fazer requisição: ",r.status_code)
    response = r.content

# descricao = response_dic['weather'][0]['description']
# temperatura = response_dic['main']['temp'] - 273.15
# print(descricao, f"{temperatura}ºC")

s3 = boto3.client(
    's3'
    , aws_access_key_id=aws_access_key_id
    , aws_secret_access_key=aws_secret_access_key
    )

bucket_name = 'bucket-teste-iam'
file_name = 'open_weather_miami.json'

s3.put_object(Bucket=bucket_name, Key=file_name, Body=response)

{'ResponseMetadata': {'RequestId': 'EKEAD0MWKWS5RQH3',
  'HostId': 'H2PniT0TTVHFg7kOpNVchZhTqP+uDYFneF8Wlj+wby1TEBEWh3Ng0KklrV9Z6L6Eep3DMBnKYgk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'H2PniT0TTVHFg7kOpNVchZhTqP+uDYFneF8Wlj+wby1TEBEWh3Ng0KklrV9Z6L6Eep3DMBnKYgk=',
   'x-amz-request-id': 'EKEAD0MWKWS5RQH3',
   'date': 'Mon, 07 Aug 2023 16:18:29 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e0ea2be95da9486bb53c8135ee14d51c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e0ea2be95da9486bb53c8135ee14d51c"',
 'ServerSideEncryption': 'AES256'}

### 5. Data Quality & Observability

#### Descrição

A qualidade dos dados é fundamental para garantir que as análises e os insights derivados sejam confiáveis. Observabilidade, por outro lado, refere-se à capacidade de monitorar e entender o comportamento dos sistemas. Para este desafio:

- Utilize o arquivo `sales_data.csv` e implemente verificações de qualidade de dados. Por exemplo:
  - Verifique se todos os IDs de usuários são únicos.
  - Confirme se os valores de vendas não são negativos.
  - Garanta que todas as entradas tenham timestamps válidos.
  - Quantidade de linhas ingeridas no banco de dados de sua escolha é igual a quantidade de linhas originais
  
- Crie métricas de observabilidade para o processo ETL que você desenvolveu anteriormente(não é necessário implementação):
  - Monitore o tempo que leva para os dados serem extraídos, transformados e carregados.
  - Implemente alertas para qualquer falha ou anomalia durante o processo ETL.
  - Descreva como você rastrearia um problema no pipeline, desde o alerta até a fonte do problema.

In [128]:
# 5.1 Utilize o arquivo `sales_data.csv` e implemente verificações de qualidade de dados.

#   - Verifique se todos os IDs de usuários são únicos.
if df_sales_raw['transaction_id'].nunique() == df_sales_raw.shape[0]:
    print("IDs de usuários são únicos.")
else:
    print("IDs de usuários não são únicos.")

#   - Confirme se os valores de vendas não são negativos.
if (df_sales_raw['sale_value'] >= 0).all():
    print("Valores de vendas não são negativos.")
else:
    print("Existem valores de vendas negativos.")

#   - Garanta que todas as entradas tenham timestamps válidos.
try:
    df_sales_raw['date'] = pd.to_datetime(df_sales_raw['date'])
    print("Timestamps válidos.")
except:
    print("Existem timestamps inválidos.")

#   - Quantidade de linhas ingeridas no banco de dados de sua escolha é igual a quantidade de linhas originais
if df_sales_refined.shape[0] == df_sales_raw.shape[0]:
    print("Quantidade de linhas após ETL é igual à quantidade de linhas originais.")
else:
    print("Quantidade de linhas após ETL não é igual à quantidade de linhas originais.")

# Monitorando o tempo do processo ETL e implementando alertas
# Você pode usar bibliotecas como time ou datetime para medir o tempo e enviar alertas

start_time = datetime.now()

#ETL
df_sales = pd.read_csv('input/sales_data.csv')
df_sales = df_sales.drop_duplicates()
df_sales = df_sales.fillna(0)
df_sales['usd_sale_value'] = df_sales['sale_value'] * 0.75

end_time = datetime.now()
execution_time = end_time - start_time

# if execution_time > YOUR_THRESHOLD:
#     send_alert("Tempo de execução do ETL excedeu o limite.")

IDs de usuários são únicos.
Valores de vendas não são negativos.
Timestamps válidos.
Quantidade de linhas após ETL é igual à quantidade de linhas originais.


### 6. Teste Unitário

#### Descrição

Os testes unitários são fundamentais para garantir a robustez e confiabilidade do código, permitindo identificar e corrigir bugs e erros antes que eles atinjam o ambiente de produção. Para este desafio:

- Escolha uma das funções ou classes que você implementou nas etapas anteriores deste teste.
- Escreva testes unitários para esta função ou classe. Os testes devem cobrir:
  - Casos padrão ou "happy path".
  - Casos de borda ou extremos.
  - Situações de erro ou exceção.
- Utilize uma biblioteca de testes de sua escolha (como `pytest`, `unittest`, etc.).
- Documente os resultados dos testes e, caso encontre falhas através dos testes, descreva como as corrigiria.

In [129]:
# Função para verificar se os valores de vendas não são negativos
def check_sales_non_negative(df):
    return (df['sales'] >= 0).all()

# Testes unitários usando o módulo unittest
class TestCheckSalesNonNegative(unittest.TestCase):
    def test_positive_values(self):
        # Caso padrão (todos os valores são não negativos)
        df_positive = pd.DataFrame({'sales': [100, 200, 150]})
        self.assertTrue(check_sales_non_negative(df_positive))

    def test_mixed_values(self):
        # Caso de borda (alguns valores negativos)
        df_mixed = pd.DataFrame({'sales': [50, 20, 300]})
        self.assertFalse(check_sales_non_negative(df_mixed))

    def test_missing_column(self):
        # Caso de erro (coluna 'sales' ausente)
        df_missing_column = pd.DataFrame({'revenue': [100, 200, 150]})
        with self.assertRaises(KeyError):
            check_sales_non_negative(df_missing_column)

    def test_empty_dataframe(self):
        # Caso de erro (DataFrame vazio)
        df_empty = pd.DataFrame({'sales': []})
        self.assertTrue(check_sales_non_negative(df_empty))

# Executar os testes
unittest.main(argv=[''], verbosity=2, exit=False)


test_empty_dataframe (__main__.TestCheckSalesNonNegative) ... ok
test_missing_column (__main__.TestCheckSalesNonNegative) ... ok
test_mixed_values (__main__.TestCheckSalesNonNegative) ... FAIL
test_positive_values (__main__.TestCheckSalesNonNegative) ... ok

FAIL: test_mixed_values (__main__.TestCheckSalesNonNegative)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_46192/3745988522.py", line 15, in test_mixed_values
    self.assertFalse(check_sales_non_negative(df_mixed))
AssertionError: True is not false

----------------------------------------------------------------------
Ran 4 tests in 0.015s

FAILED (failures=1)
